In [1]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader , DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import UnstructuredHTMLLoader

In [2]:
data_path = r"D:\Agri-Hub\rag\templates/"
dir_loader = DirectoryLoader(
    data_path,
    glob='*.html',
    loader_cls=UnstructuredHTMLLoader
)
docs = dir_loader.load()
print("HTML'S Loaded")

HTML'S Loaded


In [3]:
txt_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 200
)
inp_txt = txt_splitter.split_documents(docs)
print("Data Chunks Created")

Data Chunks Created


In [4]:
hfembeddings = HuggingFaceEmbeddings(
    model_name = "thenlper/gte-large",
    model_kwargs = {'device':'cuda'}
)
print("Embedding Created")

d:\project\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
d:\project\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
d:\project\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Embedding Created


In [5]:
db = FAISS.from_documents(inp_txt,hfembeddings)
db.save_local(r"D:\Agri-Hub\datas\faiss\Website-datas")
print("Vector Store Creation Completed and Stored Locally")

Vector Store Creation Completed and Stored Locally


d:\project\lib\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [6]:
from langchain import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import CTransformers , HuggingFaceHub
from langchain.chains import RetrievalQA

In [7]:
PROMPT_TEMPLATE = '''
With the information provided try to answer the question.If you can't find the answer dont generate irrelevant answers

Context: {context}
Question: {question}
Do provide only helpful answers

Helpful answer:
'''
INP_VARS = ['context', 'question']
custom_prompt_template = PromptTemplate(
    template = PROMPT_TEMPLATE,
    input_variables = INP_VARS
)

In [8]:
llm = CTransformers(
    model = r"D:\llama-2-7b-chat.ggmlv3.q4_1.bin",
    model_type="llama",
    max_new_tokens = 1024,
    temperature = 0.5
)

In [9]:
hfembeddings = HuggingFaceEmbeddings(
    model_name = "thenlper/gte-large",
    model_kwargs = {'device':'cpu'}
)
print("Embedding Created")

d:\project\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Embedding Created


In [10]:
vector_db = FAISS.load_local(r"D:\Agri-Hub\datas\faiss\Website-datas/", hfembeddings,allow_dangerous_deserialization=True)

In [11]:
retrieval_qa_chain = RetrievalQA.from_chain_type(
                                llm=llm,
                                chain_type="stuff",
                                retriever=vector_db.as_retriever(search_kwargs={'k': 1}),
                                return_source_documents=True,
                                chain_type_kwargs={"prompt": custom_prompt_template}
                            )

In [12]:
user_input = "what is crop insuranace ? "
prompt = {'query': user_input}
model_out = retrieval_qa_chain(prompt)
answer = model_out['result']
print(answer)

d:\project\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Crop insurance is a type of insurance that protects farmers and agricultural producers from financial losses due to crop failures, damage, or loss caused by natural disasters, diseases, or other unexpected events. It helps farmers recover their investments and continue their businesses in the face of unpredictable risks.

Additional information:
Crop insurance is often mandatory for farmers who receive government subsidies or loans to finance their agricultural activities. In some cases, it may be optional for farmers who choose to purchase coverage for their crops. The cost of crop insurance varies depending on the type of coverage, the location and size of the farm, and the historical loss experience of the area.

Helpful answer 2:
Crop insurance can provide financial protection to farmers in several ways, including:

1. Revenue loss coverage: This type of coverage provides payment to farmers when their crop yields are lower than expected due to natural disasters or other unexpected 